In [1]:
from sklearn.metrics import mean_squared_error
from module import *
from tensorflow.keras.models import load_model
import numpy as np

In [2]:
model = load_model('./train_ANN/my_model4.h5')

In [3]:
inveser_data = pd.read_csv("C:/Users/kunkun/Desktop/PythenData/train/32cases_2.csv" ,header=None)
inveser_data = np.array(inveser_data)

In [4]:
N = 200  # 粒子数量
N_chrom = 9  # 每个粒子的维度
chrom_range = np.array([[9.1, 11], [2.4, 5.5], [1.1, 3], [7.8, 10], [2.9, 4], [3.1, 5], [3.1, 5], [0.5, 3], [0.5, 3]])
iter_all = 100  # 迭代次数

In [5]:
def PSO(N, N_chrom, chrom_range, iter_all, model, target_output):
    w = 0.5  # 惯性权重
    c1 = 1.5  # 个体认知因子
    c2 = 1.5  # 群体社会因子

    # 初始化粒子群位置和速度
    particles = np.random.rand(N, N_chrom) * (chrom_range[:, 1] - chrom_range[:, 0]) + chrom_range[:, 0]
    velocities = np.random.rand(N, N_chrom) * 0.2  # 初始速度
    op_table = pd.DataFrame(particles)
    # 记录每个粒子的最佳位置和适应度值
    personal_best_positions = particles.copy()
    predicted_curves = np.array(model.predict(op_table.iloc[:,:]))
    # estimate error based on true values
    target_scale = mean_squared_error(target_output, 0*target_output)
    personal_best_fitness = [mean_squared_error(target_output, predicted_curves[k, :])/target_scale for k in range(0, predicted_curves.shape[0])]
    
    
    # 记录整个群体的最佳位置和适应度值
    global_best_index = np.argmin(personal_best_fitness)
    global_best_position = particles[global_best_index].copy()
    global_best_fitness = personal_best_fitness[global_best_index]

    # 更新粒子群
    for iter in range(1, iter_all+1):
        for i in range(N):
            # 更新速度
            op_table = pd.DataFrame(particles)
            r1, r2 = np.random.rand(N_chrom), np.random.rand(N_chrom)
            velocities[i] = w * velocities[i] + c1 * r1 * (personal_best_positions[i] - particles[i]) + \
                             c2 * r2 * (global_best_position - particles[i])

            # 更新位置
            particles[i] = particles[i] + velocities[i]

            # 确保粒子在搜索空间内
            particles[i] = np.clip(particles[i], chrom_range[:, 0], chrom_range[:, 1])
            x =  pd.DataFrame(np.zeros((1, N_chrom)))
            x.iloc[0,:] = op_table.iloc[i-1,:]
            # 计算新位置的适应度
            pre_cur = model.predict(x)
            target_output = np.reshape(target_output,(140,1))
            pre_cur = np.reshape(pre_cur,(140,1))
            new_fitness = mean_squared_error(target_output, pre_cur)/target_scale 

            # 更新个体最佳位置和适应度值
            if new_fitness < personal_best_fitness[i]:
                personal_best_fitness[i] = new_fitness
                personal_best_positions[i] = particles[i].copy()

                # 更新群体最佳位置和适应度值
                if new_fitness < global_best_fitness:
                    global_best_fitness = new_fitness
                    global_best_position = particles[i].copy()

        print(f"Iteration {iter}: Best Fitness = {global_best_fitness}")

    return global_best_position, global_best_fitness

In [51]:
best_solution = PSO(N, N_chrom, chrom_range, iter_all,model, target_output)

Iteration 1: Best Fitness = 0.0005598950770735644
Iteration 2: Best Fitness = 0.0005598950770735644
Iteration 3: Best Fitness = 0.0005464712142726477
Iteration 4: Best Fitness = 0.0005464712142726477
Iteration 5: Best Fitness = 0.0005464712142726477
Iteration 6: Best Fitness = 0.0005464712142726477
Iteration 7: Best Fitness = 0.0005464712142726477
Iteration 8: Best Fitness = 0.0005464712142726477
Iteration 9: Best Fitness = 0.0005464712142726477
Iteration 10: Best Fitness = 0.0005464712142726477
Iteration 11: Best Fitness = 0.0005464712142726477
Iteration 12: Best Fitness = 0.0005464712142726477
Iteration 13: Best Fitness = 0.0005464712142726477
Iteration 14: Best Fitness = 0.0005464712142726477
Iteration 15: Best Fitness = 0.0005464712142726477
Iteration 16: Best Fitness = 0.0005464712142726477
Iteration 17: Best Fitness = 0.0005464712142726477
Iteration 18: Best Fitness = 0.0005464712142726477
Iteration 19: Best Fitness = 0.0005464712142726477
Iteration 20: Best Fitness = 0.000546471

Iteration 161: Best Fitness = 0.0004458576506836738
Iteration 162: Best Fitness = 0.0004458576506836738
Iteration 163: Best Fitness = 0.0004458576506836738
Iteration 164: Best Fitness = 0.0004458576506836738
Iteration 165: Best Fitness = 0.0004458576506836738
Iteration 166: Best Fitness = 0.0004458576506836738
Iteration 167: Best Fitness = 0.0004458576506836738
Iteration 168: Best Fitness = 0.0004458576506836738
Iteration 169: Best Fitness = 0.0004458576506836738
Iteration 170: Best Fitness = 0.0004458576506836738
Iteration 171: Best Fitness = 0.0004458576506836738
Iteration 172: Best Fitness = 0.0004458576506836738
Iteration 173: Best Fitness = 0.0004458576506836738
Iteration 174: Best Fitness = 0.0004458576506836738
Iteration 175: Best Fitness = 0.0004458576506836738
Iteration 176: Best Fitness = 0.0004458576506836738
Iteration 177: Best Fitness = 0.0004458576506836738
Iteration 178: Best Fitness = 0.0004458576506836738
Iteration 179: Best Fitness = 0.0004458576506836738
Iteration 18

In [6]:
best_solution = np.zeros((inveser_data.shape[0],N_chrom))
best_fitness = np.zeros((1, inveser_data.shape[0]))

for i in range(inveser_data.shape[0]):
    best_solution[i, :],best_fitness[0, i] = PSO(N, N_chrom, chrom_range, iter_all,model, inveser_data[i, :])
    print(f"inveser_data {i}") 

Iteration 1: Best Fitness = 0.0016204021144632787
Iteration 2: Best Fitness = 0.0016204021144632787
Iteration 3: Best Fitness = 0.0016204021144632787
Iteration 4: Best Fitness = 0.0016204021144632787
Iteration 5: Best Fitness = 0.0016204021144632787
Iteration 6: Best Fitness = 0.0016204021144632787
Iteration 7: Best Fitness = 0.0016204021144632787
Iteration 8: Best Fitness = 0.0016204021144632787
Iteration 9: Best Fitness = 0.0016204021144632787
Iteration 10: Best Fitness = 0.0016204021144632787
Iteration 11: Best Fitness = 0.0016204021144632787
Iteration 12: Best Fitness = 0.0016204021144632787
Iteration 13: Best Fitness = 0.0016204021144632787
Iteration 14: Best Fitness = 0.0016204021144632787
Iteration 15: Best Fitness = 0.0016204021144632787
Iteration 16: Best Fitness = 0.0016204021144632787
Iteration 17: Best Fitness = 0.0016204021144632787
Iteration 18: Best Fitness = 0.0016204021144632787
Iteration 19: Best Fitness = 0.0016204021144632787
Iteration 20: Best Fitness = 0.001620402

Iteration 63: Best Fitness = 0.003970338499746203
Iteration 64: Best Fitness = 0.003970338499746203
Iteration 65: Best Fitness = 0.003970338499746203
Iteration 66: Best Fitness = 0.003970338499746203
Iteration 67: Best Fitness = 0.003970338499746203
Iteration 68: Best Fitness = 0.003970338499746203
Iteration 69: Best Fitness = 0.003970338499746203
Iteration 70: Best Fitness = 0.003970338499746203
Iteration 71: Best Fitness = 0.003970338499746203
Iteration 72: Best Fitness = 0.003970338499746203
Iteration 73: Best Fitness = 0.003970338499746203
Iteration 74: Best Fitness = 0.003970338499746203
Iteration 75: Best Fitness = 0.003970338499746203
Iteration 76: Best Fitness = 0.003970338499746203
Iteration 77: Best Fitness = 0.003970338499746203
Iteration 78: Best Fitness = 0.003970338499746203
Iteration 79: Best Fitness = 0.003970338499746203
Iteration 80: Best Fitness = 0.003970338499746203
Iteration 81: Best Fitness = 0.003970338499746203
Iteration 82: Best Fitness = 0.003970338499746203


Iteration 25: Best Fitness = 0.001645804009372983
Iteration 26: Best Fitness = 0.001645804009372983
Iteration 27: Best Fitness = 0.001645804009372983
Iteration 28: Best Fitness = 0.001645804009372983
Iteration 29: Best Fitness = 0.001645804009372983
Iteration 30: Best Fitness = 0.001645804009372983
Iteration 31: Best Fitness = 0.001645804009372983
Iteration 32: Best Fitness = 0.001645804009372983
Iteration 33: Best Fitness = 0.001645804009372983
Iteration 34: Best Fitness = 0.001645804009372983
Iteration 35: Best Fitness = 0.001645804009372983
Iteration 36: Best Fitness = 0.001645804009372983
Iteration 37: Best Fitness = 0.001645804009372983
Iteration 38: Best Fitness = 0.0010828508873449811
Iteration 39: Best Fitness = 0.0010828508873449811
Iteration 40: Best Fitness = 0.0010828508873449811
Iteration 41: Best Fitness = 0.0010828508873449811
Iteration 42: Best Fitness = 0.0010828508873449811
Iteration 43: Best Fitness = 0.0010828508873449811
Iteration 44: Best Fitness = 0.0010828508873

Iteration 88: Best Fitness = 0.001443789664632233
Iteration 89: Best Fitness = 0.001443789664632233
Iteration 90: Best Fitness = 0.001443789664632233
Iteration 91: Best Fitness = 0.001443789664632233
Iteration 92: Best Fitness = 0.001443789664632233
Iteration 93: Best Fitness = 0.001443789664632233
Iteration 94: Best Fitness = 0.001443789664632233
Iteration 95: Best Fitness = 0.001443789664632233
Iteration 96: Best Fitness = 0.001443789664632233
Iteration 97: Best Fitness = 0.001443789664632233
Iteration 98: Best Fitness = 0.001443789664632233
Iteration 99: Best Fitness = 0.001443789664632233
Iteration 100: Best Fitness = 0.001443789664632233
inveser_data 4
Iteration 1: Best Fitness = 0.0011249013803716523
Iteration 2: Best Fitness = 0.0011249013803716523
Iteration 3: Best Fitness = 0.0011249013803716523
Iteration 4: Best Fitness = 0.0011249013803716523
Iteration 5: Best Fitness = 0.0011249013803716523
Iteration 6: Best Fitness = 0.0010852941202621467
Iteration 7: Best Fitness = 0.0010

Iteration 49: Best Fitness = 0.0007438126037157003
Iteration 50: Best Fitness = 0.0007438126037157003
Iteration 51: Best Fitness = 0.0007438126037157003
Iteration 52: Best Fitness = 0.0007438126037157003
Iteration 53: Best Fitness = 0.0007438126037157003
Iteration 54: Best Fitness = 0.0007438126037157003
Iteration 55: Best Fitness = 0.0007438126037157003
Iteration 56: Best Fitness = 0.0007438126037157003
Iteration 57: Best Fitness = 0.0007438126037157003
Iteration 58: Best Fitness = 0.0007438126037157003
Iteration 59: Best Fitness = 0.0007438126037157003
Iteration 60: Best Fitness = 0.0007438126037157003
Iteration 61: Best Fitness = 0.0007438126037157003
Iteration 62: Best Fitness = 0.0007438126037157003
Iteration 63: Best Fitness = 0.0007438126037157003
Iteration 64: Best Fitness = 0.0007438126037157003
Iteration 65: Best Fitness = 0.0007438126037157003
Iteration 66: Best Fitness = 0.0007438126037157003
Iteration 67: Best Fitness = 0.0007438126037157003
Iteration 68: Best Fitness = 0.

...

Iteration 22: Best Fitness = 0.0008537549310051466
Iteration 23: Best Fitness = 0.0008537549310051466
Iteration 24: Best Fitness = 0.0008537549310051466
Iteration 25: Best Fitness = 0.0008537549310051466
Iteration 26: Best Fitness = 0.0008537549310051466
Iteration 27: Best Fitness = 0.0008537549310051466
Iteration 28: Best Fitness = 0.0008537549310051466
Iteration 29: Best Fitness = 0.0008537549310051466
Iteration 30: Best Fitness = 0.0008537549310051466
Iteration 31: Best Fitness = 0.0008153121591938396
Iteration 32: Best Fitness = 0.0008153121591938396
Iteration 33: Best Fitness = 0.0008153121591938396
Iteration 34: Best Fitness = 0.0008153121591938396
Iteration 35: Best Fitness = 0.0008153121591938396
Iteration 36: Best Fitness = 0.0008153121591938396
Iteration 37: Best Fitness = 0.0008153121591938396
Iteration 38: Best Fitness = 0.0008153121591938396
Iteration 39: Best Fitness = 0.0008153121591938396
Iteration 40: Best Fitness = 0.0008153121591938396
Iteration 41: Best Fitness = 0.

Iteration 83: Best Fitness = 0.0009248365989500204
Iteration 84: Best Fitness = 0.0009248365989500204
Iteration 85: Best Fitness = 0.0009248365989500204
Iteration 86: Best Fitness = 0.0009248365989500204
Iteration 87: Best Fitness = 0.0009248365989500204
Iteration 88: Best Fitness = 0.0009248365989500204
Iteration 89: Best Fitness = 0.0009248365989500204
Iteration 90: Best Fitness = 0.0009248365989500204
Iteration 91: Best Fitness = 0.0009248365989500204
Iteration 92: Best Fitness = 0.0009248365989500204
Iteration 93: Best Fitness = 0.0009248365989500204
Iteration 94: Best Fitness = 0.0009248365989500204
Iteration 95: Best Fitness = 0.0009248365989500204
Iteration 96: Best Fitness = 0.0009248365989500204
Iteration 97: Best Fitness = 0.0009248365989500204
Iteration 98: Best Fitness = 0.0009248365989500204
Iteration 99: Best Fitness = 0.0009248365989500204
Iteration 100: Best Fitness = 0.0009248365989500204
inveser_data 25
Iteration 1: Best Fitness = 0.0021694792986342477
Iteration 2: Be

Iteration 44: Best Fitness = 0.0012243620054642069
Iteration 45: Best Fitness = 0.0012243620054642069
Iteration 46: Best Fitness = 0.0012243620054642069
Iteration 47: Best Fitness = 0.0012243620054642069
Iteration 48: Best Fitness = 0.0012243620054642069
Iteration 49: Best Fitness = 0.0012243620054642069
Iteration 50: Best Fitness = 0.0012243620054642069
Iteration 51: Best Fitness = 0.0012243620054642069
Iteration 52: Best Fitness = 0.0012243620054642069
Iteration 53: Best Fitness = 0.0012243620054642069
Iteration 54: Best Fitness = 0.0012243620054642069
Iteration 55: Best Fitness = 0.0012243620054642069
Iteration 56: Best Fitness = 0.0012243620054642069
Iteration 57: Best Fitness = 0.0012243620054642069
Iteration 58: Best Fitness = 0.0012243620054642069
Iteration 59: Best Fitness = 0.0012243620054642069
Iteration 60: Best Fitness = 0.0012243620054642069
Iteration 61: Best Fitness = 0.0012243620054642069
Iteration 62: Best Fitness = 0.0012243620054642069
Iteration 63: Best Fitness = 0.

Iteration 5: Best Fitness = 0.0005156116481954978
Iteration 6: Best Fitness = 0.0005156116481954978
Iteration 7: Best Fitness = 0.0005156116481954978
Iteration 8: Best Fitness = 0.0005156116481954978
Iteration 9: Best Fitness = 0.0005156116481954978
Iteration 10: Best Fitness = 0.0005156116481954978
Iteration 11: Best Fitness = 0.0005156116481954978
Iteration 12: Best Fitness = 0.0005156116481954978
Iteration 13: Best Fitness = 0.0005156116481954978
Iteration 14: Best Fitness = 0.0005156116481954978
Iteration 15: Best Fitness = 0.0005156116481954978
Iteration 16: Best Fitness = 0.0005156116481954978
Iteration 17: Best Fitness = 0.0005156116481954978
Iteration 18: Best Fitness = 0.0005156116481954978
Iteration 19: Best Fitness = 0.0005156116481954978
Iteration 20: Best Fitness = 0.0005156116481954978
Iteration 21: Best Fitness = 0.0005156116481954978
Iteration 22: Best Fitness = 0.0005156116481954978
Iteration 23: Best Fitness = 0.0005156116481954978
Iteration 24: Best Fitness = 0.00051

Iteration 68: Best Fitness = 0.0002847180577070882
Iteration 69: Best Fitness = 0.0002847180577070882
Iteration 70: Best Fitness = 0.0002847180577070882
Iteration 71: Best Fitness = 0.0002847180577070882
Iteration 72: Best Fitness = 0.0002847180577070882
Iteration 73: Best Fitness = 0.0002847180577070882
Iteration 74: Best Fitness = 0.0002847180577070882
Iteration 75: Best Fitness = 0.0002847180577070882
Iteration 76: Best Fitness = 0.0002847180577070882
Iteration 77: Best Fitness = 0.0002847180577070882
Iteration 78: Best Fitness = 0.0002847180577070882
Iteration 79: Best Fitness = 0.0002847180577070882
Iteration 80: Best Fitness = 0.0002847180577070882
Iteration 81: Best Fitness = 0.0002847180577070882
Iteration 82: Best Fitness = 0.0002847180577070882
Iteration 83: Best Fitness = 0.0002847180577070882
Iteration 84: Best Fitness = 0.0002847180577070882
Iteration 85: Best Fitness = 0.0002847180577070882
Iteration 86: Best Fitness = 0.0002847180577070882
Iteration 87: Best Fitness = 0.

In [7]:
np.savetxt('./loss/'+ 'PSO_Error_2'+'.txt', np.array(best_fitness), delimiter=',')